### 1. Normalización. 
Implementa una función para limpiar y normalizar los textos de los posts 
(texto en la columna “post”). 

La versión normalizada se grabará en una nueva columna 
“clean_post”. 

Deberá realizarse una limpieza y normalización de los datos en función de las características que observes en los textos. 

Describe con detalle cada paso seguido durante el preprocesado y normalización y añade la función preprocess_post(text: str) al fichero 
core.py con la implementación final de este módulo.

### Análisis del dataframe

In [6]:
import pandas as pd

csv_file = 'reddit_database_sentiment/reddit_database_sentiment.csv'

df = pd.read_csv(
    csv_file,
    sep=';',
    engine='python',
    quotechar='"',
    encoding='utf-8',
    dtype={
    'post': str,
    }
)

print(df.head())

print(df.columns)

# print(df.dtypes)

          created_date created_timestamp  subreddit  \
0  2010-02-11 19:47:22      1265910442.0  analytics   
1  2010-03-04 20:17:26      1267726646.0  analytics   
2  2011-01-06 04:51:18      1294282278.0  analytics   
3  2011-01-19 11:45:30      1295430330.0  analytics   
4  2011-01-19 21:52:28      1295466748.0  analytics   

                                               title            author  \
0  So what do you guys all do related to analytic...              xtom   
1  Google's Invasive, non-Anonymized Ad Targeting...              xtom   
2  DotCed - Functional Web Analytics - Tagging, R...            dotced   
3            Program Details - Data Analytics Course     iqrconsulting   
4  potential job in web analytics... need to anal...  therewontberiots   

   author_created_utc                                          full_link  \
0        1.227476e+09  https://www.reddit.com/r/analytics/comments/b0...   
1        1.227476e+09  https://www.reddit.com/r/analytics/comments/b9...

In [11]:
df['post'].isna().sum()
df['post'].dropna(inplace=True)

### Peculiaridades del texto para la limpieza y normalización del mismo.

Para realizar un análisis de sentimiento sobre el texto de cada post, hay ciertas consideraciones y pasos habituales de preprocesamiento que nosotros seguiremos:

1. **Eliminación de caracteres especiales y etiquetas HTML**: 

A veces el texto podrá contener enlaces, etiquetas HTML o secuencias de caracteres no deseados.

Utilizaremos una expresión regular como ```python re.sub(r'<[^>]*>', '', texto)``` para eliminar las etiquetas HTML.

2. **Conversión a minúsculas**:

Un paso sencillo para normalizar el texto, (p.ej., "Hola" y "hola" serán considerados como la misma palabra).

3. **Eliminación de signos de puntuación**:

Para análisis de sentimiento sencillos, a menudo no se tomarán en cuenta estos signos de puntuación.

4. **Tokenización**:

Dividir el texto en palabras o frases más pequeñas, llamadas "tokens".

Para ello será utilizada la librería ```nltk``` y su función ```word_tokenize```, como en ```python nltk.word_tokenize(texto)```.

5. **Eliminación de stopwords**:

Las "stopwords" son palabras comunes que no aportan significado al texto, como "and", "the", "a", etc.

Para ello utilizaremos la librería ```nltk``` y su función ```stopwords.words('english')```, teniendo en cuenta que el idioma de los posts es el inglés.

6. **Lematización o Stemming, según la precisión que se requiera**:

Lemmatization y Stemming son técnicas de normalización de texto que buscan reducir las palabras a su raíz o lema. La primera toma en cuenta la gramática y el contexto de la palabra, mientras que la segunda es más sencilla y rápida.

Esto sirve para reducir las palabras a su forma de diccionario, agrupando diferentes inflexiones de una misma raíz, y estandarizando el texto.

Reduce el tamaño de nuestro vocabulario, reduce la 'sparsity' de nuestros datos y mejora el rendimiento de nuestro modelo.

7. **Manejo de negaciones y expresiones idiomáticas**:

Para análisis de sentimiento más avanzados, se pueden considerar expresiones idiomáticas y negaciones, como "not good".

Algunas librerías que podrían ser útiles para este análisis son ```nltk``` y ```textblob```.

8. **Corregir espacios de líneas o en blanco**:

Puede haber espacios en blanco o líneas vacías que no aporten información al texto. ```re.sub(r'\s+', ' ', texto)``` o ``` text = text.replace('\n', ' ').replace('\r', ' ')``` pueden ser útiles para corregir esto.	

9. **Consideraciones de contexto**:

Algunas expresiones en **Reddit** o redes sociales pueden ser sarcásticas, irónicas o usar emojis.

El análisis de sentimiento puede no ser preciso en estos casos, y se pueden considerar técnicas más avanzadas de procesamiento de lenguaje natural, como el uso de embeddings contextuales (p. ej., BERT).

En resumen, la correcta limpieza y estandarización del texto nos permitirá realizar un "sentiment analysis" más preciso, evitando ruidos comunes y reduciendo la complejidad de los datos antes de aplicar técnicas de NLP o modelos de Machine Learning.

In [6]:
import nltk
print(nltk.__version__)

AttributeError: partially initialized module 'nltk' has no attribute 'data' (most likely due to a circular import)

In [4]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

AttributeError: partially initialized module 'nltk' has no attribute 'data' (most likely due to a circular import)

In [24]:
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Currito\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Currito\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Currito\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [30]:
lemmatizer = WordNetLemmatizer()

def preprocess_post(text):
    if pd.isna(text):
        return ""
    
    # Remove HTML tags    
    text = re.sub(r'<[^>]*>', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters but keep spaces between words
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Remove extra whitespace
    text = ' '.join(text.split())
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    
    # Lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join with single spaces
    return ' '.join(tokens)

<h3>Prueba de la función</h3>

In [32]:
test_text = """
Check out this amazing project on machine learning! Visit https://github.com/user/repo for more details.
Thanks @datascientist for the insights. Also, shoutout to /r/MachineLearning for the support.
Contact me at 123-456-7890. #datascience #machinelearning
"""
clean_text = preprocess_post(test_text)
print("Texto original: ", test_text)
print("Texto limpio: ", clean_text)

Texto original:  
Check out this amazing project on machine learning! Visit https://github.com/user/repo for more details.
Thanks @datascientist for the insights. Also, shoutout to /r/MachineLearning for the support.
Contact me at 123-456-7890. #datascience #machinelearning

Texto limpio:  check amazing project machine learning visit details thanks insights also shoutout support contact datascience machinelearning


In [ ]:
# creacion de processed_dataset.csv con el dataframe modificado
df.to_csv('processed_dataset.csv', 
    sep=';',
    engine='python',
    quotechar='"',
    encoding='utf-8')